In [16]:
from logic.pulse_objects import PulseBlockElement
from logic.pulse_objects import PulseBlock
from logic.pulse_objects import PulseBlockEnsemble
from logic.pulse_objects import PulseSequence
import numpy as np

In [17]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
    
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
    
self = sequencegeneratorlogic

In [21]:
def generate_smart_T1_time(self, name='smart_T1', rabi_period=200e-9, mw_freq=100.0e6, mw_amp=0.25,
                      tau_start=300.0e-9, tau_incr=10e-6, num_of_points=10, mw_channel='a_ch1',
                      laser_length=3.0e-6, channel_amp=2.0, delay_length=0.7e-6, wait_time=1.0e-6,
                      sync_trig_channel='', gate_count_channel='d_ch2'):
    """

    """
    # Sanity checks
    if gate_count_channel == '':
        gate_count_channel = None
    if sync_trig_channel == '':
        sync_trig_channel = None
    err_code = self._do_channel_sanity_checks(mw_channel=mw_channel,
                                              gate_count_channel=gate_count_channel,
                                              sync_trig_channel=sync_trig_channel)
    if err_code != 0:
        return

    # get tau array for measurement ticks
    tau_array = tau_start + np.arange(num_of_points) * tau_incr
    # calculate "true" tau start value due to finite length of pi/pihalf pulses
    real_tau_start = tau_start - 3 * rabi_period / 8

    # get waiting element
    waiting_element = self._get_idle_element(wait_time, 0.0, False, gate_count_channel)
    # get laser and delay element
    laser_element, delay_element = self._get_laser_element(laser_length, 0.0, False, delay_length,
                                                           channel_amp, gate_count_channel)
    # get pi element
    pi_element = self._get_mw_element(rabi_period / 2, 0.0, mw_channel, False, mw_amp, mw_freq, 0.0, gate_count_channel)
    # get tau element
    tau_element = self._get_idle_element(real_tau_start, tau_incr, False, gate_count_channel)

    if sync_trig_channel is not None:
        # get sequence trigger element
        seqtrig_element = self._get_trigger_element(20.0e-9, 0.0, sync_trig_channel,
                                                    amp=channel_amp)
        # Create its own block out of the element
        seq_block = PulseBlock('seq_trigger', [seqtrig_element])
        # save block
        self.save_block('seq_trigger', seq_block)

    # Create element list for alternating T1 PulseBlock (waveform)
    element_list = []
    element_list.append(pi_element)
    # element_list.append(tau_element)
    # element_list.append(pi_element)
    # element_list.append(laser_element)
    # element_list.append(delay_element)
    # element_list.append(waiting_element)
    #
    # element_list.append(pi_element)
    # element_list.append(tau_element)
    # element_list.append(laser_element)
    # element_list.append(delay_element)
    # element_list.append(waiting_element)

    # Create PulseBlock object
    flip_block = PulseBlock(name, element_list)
    # save block
    self.save_block(name, flip_block)

    # Create Block list with repetitions and sequence trigger if needed.
    # remember number_of_taus=0 also counts as first round.
    block_list = [(flip_block, 1)]
    if sync_trig_channel is not None:
        block_list.append((seq_block, 0))

    # create ensemble out of the block(s)
    block_ensemble = PulseBlockEnsemble(name=name, block_list=block_list, rotating_frame=True)
    # add metadata to invoke settings later on
    block_ensemble.sample_rate = self.sample_rate
    block_ensemble.activation_config = self.activation_config
    block_ensemble.amplitude_dict = self.amplitude_dict
    block_ensemble.laser_channel = self.laser_channel
    block_ensemble.alternating = True
    block_ensemble.laser_ignore_list = []
    block_ensemble.controlled_vals_array = tau_array
    # save ensemble
    self.save_ensemble(name, block_ensemble)

    sequence_list = [(block_ensemble, {'repetitions': 2})]

    subsequence = PulseSequence(name = name, ensemble_param_list = sequence_list, rotating_frame=True)

    subsequence.sample_rate = self.sample_rate
    subsequence.activation_config = self.activation_config
    subsequence.amplitude_dict = self.amplitude_dict
    subsequence.laser_channel = self.laser_channel
    subsequence.alternating = True
    subsequence.laser_ignore_list = []

    self.save_sequence(name, subsequence)

    return subsequence

In [22]:
generate_smart_T1_time(self, name='smart_T1', rabi_period=100e-9, mw_freq=100.0e6, mw_amp=0.25,
                      tau_start=300.0e-9, tau_incr=1e-6, num_of_points=10, mw_channel='a_ch1',
                      laser_length=3.0e-6, channel_amp=2.0, delay_length=0.7e-6, wait_time=1.0e-6,
                      sync_trig_channel='', gate_count_channel='d_ch2')

In [23]:
pulsedmasterlogic.sample_sequence('smart_T1', True)